In [61]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras


In [107]:
model = "MLP"
#labels = 0
#labels = 1
labels = 2
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [108]:
csv_path = 'jena_climate_2009_2016.csv'
df = pd.read_csv(csv_path)

In [109]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [110]:
column_indices = [2, 5]
#column_indices = 5
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

In [111]:
n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

In [112]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [113]:
input_width = 6
LABEL_OPTIONS = labels

In [114]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        inputs = features[:, :-1, :]

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        labels.set_shape([None, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)
        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=input_width+1,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [115]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [116]:
MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1)
])

In [117]:
CNNmodel = keras.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

In [118]:
LSTMmodel = keras.Sequential([
    keras.layers.LSTM(units=64),
    keras.layers.Flatten(),
    keras.layers.Dense(1)
])

In [119]:
MLPmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=['MSE'])

In [120]:
CNNmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=['MSE'])

In [121]:
LSTMmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=['MSE'])

In [122]:
MLPmodel.fit(train_ds, batch_size=32, epochs=1)
#model.fit(train_ds, batch_size=32, epochs=20)

9200/9200 [==============================] - 16s 2ms/step - loss: 33.4388 - MSE: 1565.0154


In [123]:
CNNmodel.fit(train_ds, batch_size=32, epochs=1)
#CNNmodel.fit(train_ds, batch_size=32, epochs=20)

9200/9200 [==============================] - 15s 2ms/step - loss: 33.4326 - MSE: 1664.2008


In [124]:
LSTMmodel.fit(train_ds, batch_size=32, epochs=1)
#LSTMmodel.fit(train_ds, batch_size=32, epochs=20)

9200/9200 [==============================] - 38s 4ms/step - loss: 33.4256 - MSE: 1487.3052


In [125]:
test_loss, test_acc = MLPmodel.evaluate(test_ds, verbose=0)
print('\n MSE for MLP :', test_acc)
#0.22284989058971405
#1.5559452772140503
#1370.72119140625


 MSE for MLP : 1370.72119140625


In [126]:
test_loss, test_acc = CNNmodel.evaluate(test_ds, verbose=0)
print('\n MSE for CNN :', test_acc)
#0.08092997968196869
#2.0211997032165527
#1455.0245361328125


 MSE for CNN : 1455.0245361328125


In [127]:
test_loss, test_acc = LSTMmodel.evaluate(test_ds, verbose=0)
print('\n MSE for LSTM :', test_acc)
#0.2179471105337143
#2.320437431335449
#1272.8814697265625


 MSE for LSTM : 1272.8814697265625


In [83]:
run_model = tf.function(lambda x: MLPmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2],tf.float32))
MLPmodel.save('MLP', signatures=concrete_func)

INFO:tensorflow:Assets written to: MLP\assets


INFO:tensorflow:Assets written to: MLP\assets


In [84]:
run_model = tf.function(lambda x: CNNmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2],tf.float32))
CNNmodel.save('CNN', signatures=concrete_func)

INFO:tensorflow:Assets written to: CNN\assets


INFO:tensorflow:Assets written to: CNN\assets


In [85]:
run_model = tf.function(lambda x: LSTMmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2],tf.float32))
LSTMmodel.save('LSTM', signatures=concrete_func)

INFO:tensorflow:Assets written to: LSTM\assets


INFO:tensorflow:Assets written to: LSTM\assets
